In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.model_selection import train_test_split # split data into training and validation data, for both features and target
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
import xgboost as XGB
from sklearn.impute import SimpleImputer
# data normalization with sklearn
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import f1_score
from sklearn.model_selection import cross_val_score 
from sklearn.compose import ColumnTransformer #cross validation
from sklearn.pipeline import Pipeline
import seaborn as sns
import sklearn
import matplotlib.pyplot as plt
%matplotlib inline

# classification Logistic Regression,k-Nearest Neighbors,Decisiontrees,SupportVectorMachine,Naive Bayes
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/company-bankruptcy-prediction/data.csv


In [2]:
# Load data
company_bankruptcy = pd.read_csv('/kaggle/input/company-bankruptcy-prediction/data.csv') 

**Exploratory Data Analysis**

In [3]:
company_bankruptcy.head()

,Bankrupt?,ROA(C) before interest and depreciation before interest,ROA(A) before interest and % after tax,ROA(B) before interest and depreciation after tax,Operating Gross Margin,Realized Sales Gross Margin,Operating Profit Rate,Pre-tax net Interest Rate,After-tax net Interest Rate,Non-industry income and expenditure/revenue,...,Net Income to Total Assets,Total assets to GNP price,No-credit Interval,Gross Profit to Sales,Net Income to Stockholder's Equity,Liability to Equity,Degree of Financial Leverage (DFL),Interest Coverage Ratio (Interest expense to EBIT),Net Income Flag,Equity to Liability
0,1,0.370594,0.424389,0.405750,0.601457,0.601457,0.998969,0.796887,0.808809,0.302646,...,0.716845,0.009219,0.622879,0.601453,0.827890,0.290202,0.026601,0.564050,1,0.016469
1,1,0.464291,0.538214,0.516730,0.610235,0.610235,0.998946,0.797380,0.809301,0.303556,...,0.795297,0.008323,0.623652,0.610237,0.839969,0.283846,0.264577,0.570175,1,0.020794
2,1,0.426071,0.499019,0.472295,0.601450,0.601364,0.998857,0.796403,0.808388,0.302035,...,0.774670,0.040003,0.623841,0.601449,0.836774,0.290189,0.026555,0.563706,1,0.016474
3,1,0.399844,0.451265,0.457733,0.583541,0.583541,0.998700,0.796967,0.808966,0.303350,...,0.739555,0.003252,0.622929,0.583538,0.834697,0.281721,0.026697,0.564663,1,0.023982
4,1,0.465022,0.538432,0.522298,0.598783,0.598783,0.998973,0.797366,0.809304,0.303475,...,0.795016,0.003878,0.623521,0.598782,0.839973,0.278514,0.024752,0.575617,1,0.035490


In [4]:
company_bankruptcy.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6819 entries, 0 to 6818
Data columns (total 96 columns):
 #   Column                                                    Non-Null Count  Dtype  
---  ------                                                    --------------  -----  
 0   Bankrupt?                                                 6819 non-null   int64  
 1    ROA(C) before interest and depreciation before interest  6819 non-null   float64
 2    ROA(A) before interest and % after tax                   6819 non-null   float64
 3    ROA(B) before interest and depreciation after tax        6819 non-null   float64
 4    Operating Gross Margin                                   6819 non-null   float64
 5    Realized Sales Gross Margin                              6819 non-null   float64
 6    Operating Profit Rate                                    6819 non-null   float64
 7    Pre-tax net Interest Rate                                6819 non-null   float64
 8    After-tax net Int

In [5]:
company_bankruptcy.describe()

,Bankrupt?,ROA(C) before interest and depreciation before interest,ROA(A) before interest and % after tax,ROA(B) before interest and depreciation after tax,Operating Gross Margin,Realized Sales Gross Margin,Operating Profit Rate,Pre-tax net Interest Rate,After-tax net Interest Rate,Non-industry income and expenditure/revenue,...,Net Income to Total Assets,Total assets to GNP price,No-credit Interval,Gross Profit to Sales,Net Income to Stockholder's Equity,Liability to Equity,Degree of Financial Leverage (DFL),Interest Coverage Ratio (Interest expense to EBIT),Net Income Flag,Equity to Liability
count,6819.000000,6819.000000,6819.000000,6819.000000,6819.000000,6819.000000,6819.000000,6819.000000,6819.000000,6819.000000,...,6819.000000,6.819000e+03,6819.000000,6819.000000,6819.000000,6819.000000,6819.000000,6819.000000,6819.0,6819.000000
mean,0.032263,0.505180,0.558625,0.553589,0.607948,0.607929,0.998755,0.797190,0.809084,0.303623,...,0.807760,1.862942e+07,0.623915,0.607946,0.840402,0.280365,0.027541,0.565358,1.0,0.047578
std,0.176710,0.060686,0.065620,0.061595,0.016934,0.016916,0.013010,0.012869,0.013601,0.011163,...,0.040332,3.764501e+08,0.012290,0.016934,0.014523,0.014463,0.015668,0.013214,0.0,0.050014
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.0,0.000000
25%,0.000000,0.476527,0.535543,0.527277,0.600445,0.600434,0.998969,0.797386,0.809312,0.303466,...,0.796750,9.036205e-04,0.623636,0.600443,0.840115,0.276944,0.026791,0.565158,1.0,0.024477
50%,0.000000,0.502706,0.559802,0.552278,0.605997,0.605976,0.999022,0.797464,0.809375,0.303525,...,0.810619,2.085213e-03,0.623879,0.605998,0.841179,0.278778,0.026808,0.565252,1.0,0.033798
75%,0.000000,0.535563,0.589157,0.584105,0.613914,0.613842,0.999095,0.797579,0.809469,0.303585,...,0.826455,5.269777e-03,0.624168,0.613913,0.842357,0.281449,0.026913,0.565725,1.0,0.052838
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,9.820000e+09,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.0,1.000000


In [6]:
company_bankruptcy.isnull().sum()
# no null value

Bankrupt?                                                   0
 ROA(C) before interest and depreciation before interest    0
 ROA(A) before interest and % after tax                     0
 ROA(B) before interest and depreciation after tax          0
 Operating Gross Margin                                     0
                                                           ..
 Liability to Equity                                        0
 Degree of Financial Leverage (DFL)                         0
 Interest Coverage Ratio (Interest expense to EBIT)         0
 Net Income Flag                                            0
 Equity to Liability                                        0
Length: 96, dtype: int64

In [7]:
# Choose target and features
y = company_bankruptcy['Bankrupt?']

X = company_bankruptcy.copy()
X.drop(['Bankrupt?'], axis = 1, inplace = True)

In [8]:
X.head()

,ROA(C) before interest and depreciation before interest,ROA(A) before interest and % after tax,ROA(B) before interest and depreciation after tax,Operating Gross Margin,Realized Sales Gross Margin,Operating Profit Rate,Pre-tax net Interest Rate,After-tax net Interest Rate,Non-industry income and expenditure/revenue,Continuous interest rate (after tax),...,Net Income to Total Assets,Total assets to GNP price,No-credit Interval,Gross Profit to Sales,Net Income to Stockholder's Equity,Liability to Equity,Degree of Financial Leverage (DFL),Interest Coverage Ratio (Interest expense to EBIT),Net Income Flag,Equity to Liability
0,0.370594,0.424389,0.405750,0.601457,0.601457,0.998969,0.796887,0.808809,0.302646,0.780985,...,0.716845,0.009219,0.622879,0.601453,0.827890,0.290202,0.026601,0.564050,1,0.016469
1,0.464291,0.538214,0.516730,0.610235,0.610235,0.998946,0.797380,0.809301,0.303556,0.781506,...,0.795297,0.008323,0.623652,0.610237,0.839969,0.283846,0.264577,0.570175,1,0.020794
2,0.426071,0.499019,0.472295,0.601450,0.601364,0.998857,0.796403,0.808388,0.302035,0.780284,...,0.774670,0.040003,0.623841,0.601449,0.836774,0.290189,0.026555,0.563706,1,0.016474
3,0.399844,0.451265,0.457733,0.583541,0.583541,0.998700,0.796967,0.808966,0.303350,0.781241,...,0.739555,0.003252,0.622929,0.583538,0.834697,0.281721,0.026697,0.564663,1,0.023982
4,0.465022,0.538432,0.522298,0.598783,0.598783,0.998973,0.797366,0.809304,0.303475,0.781550,...,0.795016,0.003878,0.623521,0.598782,0.839973,0.278514,0.024752,0.575617,1,0.035490


In [9]:
train_X, val_X, train_y, val_y = train_test_split(X, y,random_state = 0, test_size = 0.2, shuffle=True, stratify = y)

In [10]:
# fit scaler on training data
norm = MinMaxScaler().fit(train_X)

# transform training data X_train_norm
train_X = norm.transform(train_X)

# transform testing dataabs X_val_norm
val_X = norm.transform(val_X)

In [11]:
#Decision Tree classifier 0.2677
def get_score():
    my_pipeline = Pipeline(steps=[
        ('preprocessor', SimpleImputer()),
        ('model', DecisionTreeClassifier(random_state=0))
    ])
    scores = cross_val_score(my_pipeline, X, y,
                                  cv=3,
                                  scoring='f1')
    return scores.mean()
get_score()

0.26771030477060626

In [12]:
#Decision Tree classifier 0.2677, minmax perfomed better than simpleimputer 0.27440
def get_scored():
    my_pipeline = Pipeline(steps=[
        ('preprocessor', MinMaxScaler()),
        ('model', DecisionTreeClassifier(random_state=0))
    ])
    score = cross_val_score(my_pipeline, X, y,
                                  cv=45, n_jobs = -1,
                                  scoring='f1')
    return score.mean()
get_scored()

0.3838648686481504

In [13]:
# svc, no difference on this
def get_svc():
    my_pipeline = Pipeline(steps=[
        ('preprocessor', SimpleImputer()),
        ('model', SVC(random_state=0))
    ])
    scores = cross_val_score(my_pipeline, X, y,
                                  cv=45,
                                  scoring='f1')
    return scores.mean()
get_svc()

0.0

In [14]:
# Naive bayes
def get_gnb():
    my_pipeline = Pipeline(steps=[
        ('preprocessor', SimpleImputer()),
        ('model', GaussianNB())
    ])
    scores = cross_val_score(my_pipeline, X, y,
                                  cv=3,
                                  scoring='f1')
    return scores.mean()
get_gnb()

0.06245986630938107

In [15]:
# Naive bayes it also performed better on this, the higher the cv, the better the model 0.185219 50
def get_gnbs():
    my_pipeline = Pipeline(steps=[
        ('preprocessor', MinMaxScaler()),
        ('model', GaussianNB())
    ])
    scores = cross_val_score(my_pipeline, train_X, train_y,
                                  cv=200, n_jobs = -1,
                                  scoring='f1')
    return scores.mean()
get_gnbs()
#0.19823376855393035

/opt/conda/lib/python3.7/site-packages/sklearn/model_selection/_split.py:672: UserWarning: The least populated class in y has only 176 members, which is less than n_splits=200.
  % (min_groups, self.n_splits)), UserWarning)


0.1064819307799184

In [16]:
# logistic  
def get_log():
    my_pipeline = Pipeline(steps=[
        ('preprocessor', MinMaxScaler()),
        ('model', LogisticRegression(solver='liblinear'))
    ])
    scores = cross_val_score(my_pipeline, X, y,
                                  cv=2, n_jobs = -1,
                                  scoring='f1')
    return scores.mean()
get_log()
#0.19976635514018692

0.1658730158730159

In [17]:
# logistic regression , l1_ratio = 0.5 0.043
log_model = LogisticRegression(solver='liblinear')
# fit model
log_model.fit(train_X, train_y)
# predict on test set
log_pred = log_model.predict(val_X)
print('F-measure: %.3f' % f1_score(val_y, log_pred))

F-measure: 0.214


In [18]:
pd.DataFrame(log_pred).head()

,0
0,0
1,0
2,0
3,0
4,0


In [19]:
# naive bayes 0.063
gnb2_model = GaussianNB()


# fit model
gnb2_model.fit(train_X, train_y)

# predict on test set
gnb2_pred = gnb2_model.predict(val_X)
print('F-measure: %.3f' % f1_score(val_y, gnb2_pred))

# without encoding 0.059

F-measure: 0.107


In [20]:
# svc 0.043
svc2_model = SVC()
# fit model
svc2_model.fit(train_X, train_y)
# predict on test set
svc2_pred = log_model.predict(val_X)
print('F-measure: %.3f' % f1_score(val_y, svc2_pred))

F-measure: 0.214


In [21]:
# Decision Tree 0.319
dtc2_model = DecisionTreeClassifier()
# fit model
dtc2_model.fit(train_X, train_y)
# predict on test set
dtc2_pred = dtc2_model.predict(val_X)
print('F-measure: %.3f' % f1_score(val_y, dtc2_pred))

F-measure: 0.260


In [22]:
# 0.333, 0.388
xgb_model = XGB.XGBClassifier()
# fit model
xgb_model.fit(train_X, train_y)
# predict on test set
xgb_pred = xgb_model.predict(val_X)
print('F-measure: %.3f' % f1_score(val_y, xgb_pred))

[05:53:10] WARNING: ../src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


/opt/conda/lib/python3.7/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


F-measure: 0.388


In [23]:
# xgb
def get_xgb():
    my_pipeline = Pipeline(steps=[
        ('preprocessor', MinMaxScaler()),
        ('model',XGB.XGBClassifier())
    ])
    scores = cross_val_score(my_pipeline, train_X, train_y,
                                  cv=2, n_jobs = -1,
                                  scoring='f1')
    return scores.mean()
get_xgb()
# XGB performed better than all using cross_val_score

0.3979375991538868